# Análise comparativa de modelos

----
Neste notebook iremos fazer a limpeza e transformação de dados do dataset [Titanic - Machine Learning From Disaster](https://www.kaggle.com/c/titanic/overview), além de realizar uma análise comparativa de modelos.

----

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression

## 1. Obtenção de dados

Vamos obter novamente os arquivos brutos de dados e o dicionário criado a partir desses dados antes de iniciar o pré-processamento.

In [2]:
df = pd.read_csv("../data/raw/data.csv")
dicionario = pd.read_csv("../data/external/dictionary.csv")
display(df)
dicionario

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


,variavel,descricao,tipo,subtipo
0,survived,Indica se a pessoa sobreviveu ou não,quantitativa,discreta
1,pclass,Classe do passageiro no navio,qualitativa,ordinal
2,sex,Sexo do passageiro,qualitativa,nominal
3,age,Idade do passageiro,quantitativa,contínua
4,sibsp,Quantidade de irmãos ou cônjuges a bordo do navio,quantitativa,discreta
5,parch,Quantidade de pais ou filhos a bordo do navio,quantitativa,discreta
6,fare,Valor da tarifa do passageiro,quantitativa,contínua
7,embarked,Porto de embarcação do passageiro,qualitativa,nominal
8,class,Classe do passageiro no navio,qualitativa,ordinal
9,who,"Indica se é homem, mulher ou criança",qualitativa,nominal


## 2. Preparação de dados

Nesta etapa iremos realizar a imputação de dados faltantes na variável `age` por meio do método *hot-deck*, imputação de dados nas variáveis `embarked` e `embark_town` pela moda dessas variáveis, eliminaremos a coluna `deck`, pois há muitos dados faltantes e essa variável não será importante para a predição, e realizaremos a codificação e normalização dos dados.

In [3]:
variavel_alvo = 'alive'
variavel_excluida = 'deck'
variaveis_nominais = (dicionario.query("subtipo == 'nominal' and variavel != @variavel_alvo").variavel.to_list())
variaveis_ordinais = (dicionario.query("subtipo == 'ordinal' and variavel != @variavel_excluida").variavel.to_list())
variaveis_discretas = (dicionario.query("subtipo == 'discreta'").variavel.to_list())
variaveis_continuas = (dicionario.query("subtipo == 'contínua'").variavel.to_list())

X = df.drop(columns=[variavel_alvo, variavel_excluida], axis=1) # entradas
y = df[variavel_alvo] # saída

0       no
1      yes
2      yes
3      yes
4       no
      ... 
886     no
887    yes
888     no
889    yes
890     no
Name: alive, Length: 891, dtype: object

In [4]:
# removendo a coluna 'deck'
df.drop(columns=['deck'], inplace=True)
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,Cherbourg,yes,True


In [5]:
'''
# outra forma de fazer os tratamentos
# tratamento de dados faltantes em age
k_imputer = KNNImputer(n_neighbors=10).fit(df[variaveis_quantitativas])
df[variaveis_quantitativas] = k_imputer.transform(df[variaveis_quantitativas])

# tratamento de dados faltantes nas variáveis qualitativas
s_imputer = SimpleImputer(strategy='most_frequent').fit(df[variaveis_qualitativas])
df[variaveis_qualitativas] = s_imputer.transform(df[variaveis_qualitativas])
print(df.isna().sum())
'''

"\n# outra forma de fazer os tratamentos\n# tratamento de dados faltantes em age\nk_imputer = KNNImputer(n_neighbors=10).fit(df[variaveis_quantitativas])\ndf[variaveis_quantitativas] = k_imputer.transform(df[variaveis_quantitativas])\n\n# tratamento de dados faltantes nas variáveis qualitativas\ns_imputer = SimpleImputer(strategy='most_frequent').fit(df[variaveis_qualitativas])\ndf[variaveis_qualitativas] = s_imputer.transform(df[variaveis_qualitativas])\nprint(df.isna().sum())\n"

In [6]:
preprocess_continuas = Pipeline([
    ('missing', KNNImputer(n_neighbors=10)), # tratamento dados faltantes
    # dados quantitativos não precisam ser transformados
    ('normalization', StandardScaler())  # normalização
])

preprocess_discretas = Pipeline([
    # não há dados faltantes nas variáveis discretas
    # dados quantitativos não precisam ser transformados
    ('normalization', StandardScaler()) # normalização
])

preprocess_nominais = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # tratamento dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), # transformação de dados
    ('normalization', StandardScaler())  # normalização
])

preprocess_ordinais = Pipeline([
    # não há dados faltantes nas variáveis ordinais
    ('encoding', OrdinalEncoder()), # codificação de variáveis
    ('normalization', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('continuas', preprocess_continuas, variaveis_continuas),
    ('discretas', preprocess_discretas, variaveis_discretas),
    ('nominais', preprocess_nominais, variaveis_nominais),
    ('ordinais', preprocess_ordinais, variaveis_ordinais)
])

In [7]:
preprocessor.fit(X)

ColumnTransformer(transformers=[('continuas',
                                 Pipeline(steps=[('missing',
                                                  KNNImputer(n_neighbors=10)),
                                                 ('normalization',
                                                  StandardScaler())]),
                                 ['age', 'fare']),
                                ('discretas',
                                 Pipeline(steps=[('normalization',
                                                  StandardScaler())]),
                                 ['survived', 'sibsp', 'parch']),
                                ('nominais',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False)),
                                                 ('normalization',
                                                  StandardScaler())]),
                                 ['sex', 'embarked', 'who', 'adult_male',
                                  'embark_town', 'alone']),
                                ('ordinais',
                                 Pipeline(steps=[('encoding', OrdinalEncoder()),
                                                 ('normalization',
                                                  StandardScaler())]),
                                 ['pclass', 'class'])])

## 3. Seleção de modelos

### 3.1. Resultados gerais

### 3.2. Persistência do modelo